In [6]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

def load_images_and_labels_from_csv(csv_path, images_dir, image_size=(32, 32), num_classes=43):
    images = []
    labels = []
    data = pd.read_csv(csv_path)
    for idx, row in data.iterrows():
        image_path = os.path.join(images_dir, row['Path'].replace('/', os.sep))
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Resimleri RGB formatında oku
        if image is not None:
            image = cv2.resize(image, image_size)  # Resimleri belirtilen boyuta getir
            images.append(image)
            labels.append(int(row['ClassId']))
        else:
            print(f"Failed to load image at path: {image_path}")
    images = np.array(images).astype('float32') / 255.0  # Normalizasyon
    labels = to_categorical(np.array(labels), num_classes)  # Etiketleri one-hot encoding'e çevir
    return images, labels

# Güncellenmiş veri yolları ve CSV dosya yolları
train_dir = r'C:\Users\hakan\Desktop\KablosuzAg\Proje\GTSRB'
train_csv = r'C:\Users\hakan\Desktop\KablosuzAg\Proje\GTSRB\Train.csv'
valid_dir = r'C:\Users\hakan\Desktop\KablosuzAg\Proje\GTSRB'
valid_csv = r'C:\Users\hakan\Desktop\KablosuzAg\Proje\GTSRB\Meta.csv'
test_dir = r'C:\Users\hakan\Desktop\KablosuzAg\Proje\GTSRB'
test_csv = r'C:\Users\hakan\Desktop\KablosuzAg\Proje\GTSRB\Test.csv'

# Verileri CSV dosyaları kullanarak yükle
X_train, y_train = load_images_and_labels_from_csv(train_csv, train_dir)
X_valid, y_valid = load_images_and_labels_from_csv(valid_csv, valid_dir)
X_test, y_test = load_images_and_labels_from_csv(test_csv, test_dir)

# Doğrulayalım
print("Training data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_valid.shape, y_valid.shape)
print("Test data shape:", X_test.shape, y_test.shape)


KeyboardInterrupt: 

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Özelleştirilmiş CNN modelini oluşturalım
def create_custom_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Modeli eğitim ve test verileriyle kullanacak şekilde özelleştir
input_shape = X_train.shape[1:]  # Giriş veri şekli
num_classes = y_train.shape[1]  # Toplam sınıf sayısı

custom_model = create_custom_model(input_shape, num_classes)

# Modeli eğitelim
history_custom = custom_model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_valid, y_valid)
)

# Model performansını değerlendirelim
test_loss_custom, test_accuracy_custom = custom_model.evaluate(X_test, y_test)
print("Custom CNN Test Loss:", test_loss_custom)
print("Custom CNN Test Accuracy:", test_accuracy_custom)

# Modelin test verileri üzerindeki tahminlerini al
y_pred = custom_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Çok sınıflı sınıflandırma için en yüksek olasılıklı sınıfı seç
y_true = np.argmax(y_test, axis=1)  # Gerçek etiketleri one-hot'dan çevir

# Sınıflandırma raporunu hesapla ve yazdır
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))
# Sınıflandırma raporunu hesapla
class_report = classification_report(y_true, y_pred_classes, output_dict=True)

# Dictionary'den DataFrame oluştur
df_class_report = pd.DataFrame(class_report).transpose()

# DataFrame'i ekrana yazdır
print("Classification Report:")
print(df_class_report)
# Karışıklık matrisini hesapla
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Karışıklık matrisini ısı haritası olarak görselleştir (büyük boyut)
plt.figure(figsize=(16, 14))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True)

# Eksen etiketleri, başlık ekleyelim
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 15, 15, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │        11,051 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 666,699 (2.54 MB)

 Trainable params: 666,699 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32,), dtype=float32). Expected shape (None, 32, 32, 3), but input has incompatible shape (32,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(32,), dtype=float32)
  • training=True
  • mask=None

In [ ]:
# Transfer öğrenme modelini oluşturalım (MobileNetV2)
from tensorflow.keras.applications import MobileNetV2, Xception

# Transfer öğrenme modelini oluşturalım (MobileNetV2)
base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
transfer_mobilenet = Sequential([
    base_model_mobilenet,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
transfer_mobilenet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitelim (MobileNetV2)
history_mobilenet = transfer_mobilenet.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_valid, y_valid)
)

# Model performansını değerlendirelim (MobileNetV2)
test_loss_mobilenet, test_accuracy_mobilenet = transfer_mobilenet.evaluate(X_test, y_test)
print("Transfer Learning (MobileNetV2) Test Loss:", test_loss_mobilenet)
print("Transfer Learning (MobileNetV2) Test Accuracy:", test_accuracy_mobilenet)

# Modelin test verileri üzerindeki tahminlerini al
y_pred = custom_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Çok sınıflı sınıflandırma için en yüksek olasılıklı sınıfı seç
y_true = np.argmax(y_test, axis=1)  # Gerçek etiketleri one-hot'dan çevir

# Sınıflandırma raporunu hesapla ve yazdır
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))
# Sınıflandırma raporunu hesapla
class_report = classification_report(y_true, y_pred_classes, output_dict=True)

# Dictionary'den DataFrame oluştur
df_class_report = pd.DataFrame(class_report).transpose()

# DataFrame'i ekrana yazdır
print("Classification Report:")
print(df_class_report)
# Karışıklık matrisini hesapla
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Karışıklık matrisini ısı haritası olarak görselleştir (büyük boyut)
plt.figure(figsize=(16, 14))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True)

# Eksen etiketleri, başlık ekleyelim
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Transfer öğrenme modelini oluşturalım (Xception)
base_model_xception = Xception(weights='imagenet', include_top=False, input_shape=(71, 71, 3))

transfer_xception = Sequential([
    base_model_xception,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

transfer_xception.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitelim (Xception)
history_xception = transfer_xception.fit(
    tf.image.resize(X_train, (71, 71)), y_train,
    epochs=10,
    batch_size=32,
    validation_data=(tf.image.resize(X_valid, (71, 71)), y_valid)
)

# Model performansını değerlendirelim (Xception)
test_loss_xception, test_accuracy_xception = transfer_xception.evaluate(tf.image.resize(X_test, (71, 71)), y_test)
print("Transfer Learning (Xception) Test Loss:", test_loss_xception)
print("Transfer Learning (Xception) Test Accuracy:", test_accuracy_xception)


In [ ]:

# Tahminleri alalım
y_pred_custom = np.argmax(custom_model.predict(X_test), axis=1)
y_pred_mobilenet = np.argmax(transfer_mobilenet.predict(X_test), axis=1)
y_pred_xception = np.argmax(transfer_xception.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

# Metrikleri hesaplayalım
print("Custom CNN Classification Report:")
print(classification_report(y_true, y_pred_custom))

print("MobileNetV2 Transfer Learning Classification Report:")
print(classification_report(y_true, y_pred_mobilenet))

print("Xception Transfer Learning Classification Report:")
print(classification_report(y_true, y_pred_xception))

# Confusion matrix'i görselleştirelim
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

plot_confusion_matrix(y_true, y_pred_custom, title='Custom CNN Confusion Matrix')
plot_confusion_matrix(y_true, y_pred_mobilenet, title='MobileNetV2 Transfer Learning Confusion Matrix')
plot_confusion_matrix(y_true, y_pred_xception, title='Xception Transfer Learning Confusion Matrix')